This is for testing out some things regarding the posterior distribution before diving
deep into the theory.

In [1]:
import torch
from botorch import fit_gpytorch_model
from botorch.models import SingleTaskGP
from botorch.models.transforms import Standardize
from gpytorch import ExactMarginalLogLikelihood

In [2]:
train_X = torch.linspace(0, 1, 4).view(-1, 1)
train_Y = torch.randn(4, 1)
model = SingleTaskGP(
    train_X, train_Y, outcome_transform=Standardize(m=1)
)
mll = ExactMarginalLogLikelihood(model.likelihood, model)
_ = fit_gpytorch_model(mll)

In [3]:
sigma_0 = model.forward(train_X).covariance_matrix
sigma_1 = model(train_X).covariance_matrix

m2 = model.condition_on_observations(train_X[:1], train_Y[:1])
sigma_2 = m2(train_X).covariance_matrix
sigma_2_0 = m2.forward(train_X).covariance_matrix

/home/saitcakmak/anaconda3/envs/contextual_rs/lib/python3.8/site-packages/gpytorch/models/exact_gp.py:273: GPInputWarning: The input matches the stored training data. Did you forget to call model.train()?
  warnings.warn(


In [4]:
print(f"Prior: {sigma_0}")
print(f"Posterior: {sigma_1}")
print(f"Posterior n+1: {sigma_2}")
print(f"Prior n+1: {sigma_2_0}")

Prior: tensor([[1.7682, 1.1499, 0.4485, 0.1379],
        [1.1499, 1.7682, 1.1499, 0.4485],
        [0.4485, 1.1499, 1.7682, 1.1499],
        [0.1379, 0.4485, 1.1499, 1.7682]], grad_fn=<MulBackward0>)
Posterior: tensor([[ 0.0623,  0.0034, -0.0013,  0.0003],
        [ 0.0034,  0.0596,  0.0043, -0.0013],
        [-0.0013,  0.0043,  0.0596,  0.0034],
        [ 0.0003, -0.0013,  0.0034,  0.0623]], grad_fn=<AddmmBackward>)
Posterior n+1: tensor([[ 0.0322,  0.0017, -0.0006,  0.0001],
        [ 0.0017,  0.0595,  0.0043, -0.0013],
        [-0.0006,  0.0043,  0.0596,  0.0034],
        [ 0.0001, -0.0013,  0.0034,  0.0623]], grad_fn=<AddmmBackward>)
Prior n+1: tensor([[1.7682, 1.1499, 0.4485, 0.1379],
        [1.1499, 1.7682, 1.1499, 0.4485],
        [0.4485, 1.1499, 1.7682, 1.1499],
        [0.1379, 0.4485, 1.1499, 1.7682]], grad_fn=<MulBackward0>)


In [5]:
def get_rho(i, j, covar):
    rho_sq = covar[i, j].pow(2) / (covar[i, i] * covar[j, j])
    return rho_sq.sqrt()

print(f"prior rho: {get_rho(0, 1, sigma_0)}")
print(f"post rho: {get_rho(0, 1, sigma_1)}")
print(f"n+1 rho: {get_rho(0, 1, sigma_2)}")

prior rho: 0.6503499746322632
post rho: 0.055445101112127304
n+1 rho: 0.0398750826716423
